In [1]:
import xml.etree.ElementTree as ET

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = []
    
    for obj in root.findall('object'):
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        annotations.append((name, (xmin, ymin, xmax, ymax)))
    
    return annotations

In [4]:
import cv2
import os

def preprocess_images(image_dir, annotations_dir, output_dir):
    for xml_file in os.listdir(annotations_dir):
        if xml_file.endswith('.xml'):
            image_file = xml_file.replace('.xml', '.png')  # or .png depending on your images
            image_path = os.path.join(image_dir, image_file)
            annotations_path = os.path.join(annotations_dir, xml_file)
            
            image = cv2.imread(image_path)
            annotations = parse_xml(annotations_path)
            
            for idx, (label, (xmin, ymin, xmax, ymax)) in enumerate(annotations):
                roi = image[ymin:ymax, xmin:xmax]
                output_label_dir = os.path.join(output_dir, label)
                os.makedirs(output_label_dir, exist_ok=True)
                output_path = os.path.join(output_label_dir, f'{os.path.splitext(image_file)[0]}_{idx}.jpg')
                cv2.imwrite(output_path, roi)


In [7]:
import xml.etree.ElementTree as ET
import cv2
import os

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = []
    
    for obj in root.findall('object'):
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        annotations.append((name, (xmin, ymin, xmax, ymax)))
    
    return annotations

def preprocess_images(image_dir, annotations_dir, output_dir):
    for xml_file in os.listdir(annotations_dir):
        if xml_file.endswith('.xml'):
            image_file = xml_file.replace('.xml', '.png') 
            image_path = os.path.join(image_dir, image_file)
            annotations_path = os.path.join(annotations_dir, xml_file)
            
            if not os.path.exists(image_path):
                print(f"Image file not found: {image_path}")
                continue
            
            image = cv2.imread(image_path)
            
            if image is None:
                print(f"Failed to load image: {image_path}")
                continue
            
            height, width, _ = image.shape
            annotations = parse_xml(annotations_path)
            
            for idx, (label, (xmin, ymin, xmax, ymax)) in enumerate(annotations):
                xmin = max(0, xmin)
                ymin = max(0, ymin)
                xmax = min(width, xmax)
                ymax = min(height, ymax)
                
                if xmax <= xmin or ymax <= ymin:
                    print(f"Invalid bounding box for {image_file}: {(xmin, ymin, xmax, ymax)}")
                    continue
                
                roi = image[ymin:ymax, xmin:xmax]
                
                if roi.size == 0:
                    print(f"Empty ROI for {image_file}: {(xmin, ymin, xmax, ymax)}")
                    continue
                
                output_label_dir = os.path.join(output_dir, label.replace(" ", "_"))
                os.makedirs(output_label_dir, exist_ok=True)
                output_path = os.path.join(output_label_dir, f'{os.path.splitext(image_file)[0]}_{idx}.jpg')
                cv2.imwrite(output_path, roi)

image_dir = r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\images"
annotations_dir = r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\annotations"
output_dir = r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\Final"

preprocess_images(image_dir, annotations_dir, output_dir)


Invalid bounding box for BikesHelmets103.png: (149282, 10520, 400, 263)
Invalid bounding box for BikesHelmets140.png: (192641, 2403, 400, 267)
Invalid bounding box for BikesHelmets205.png: (112047, 2028, 507, 338)
Invalid bounding box for BikesHelmets279.png: (57200, 2400, 400, 400)
Invalid bounding box for BikesHelmets326.png: (34200, 3200, 300, 400)
Invalid bounding box for BikesHelmets343.png: (249264, 26093, 400, 269)
Invalid bounding box for BikesHelmets441.png: (112000, 3000, 400, 300)
Invalid bounding box for BikesHelmets444.png: (52000, 1272, 200, 159)
Invalid bounding box for BikesHelmets530.png: (104918, 4272, 400, 267)
Invalid bounding box for BikesHelmets616.png: (214800, 50400, 300, 200)
Invalid bounding box for BikesHelmets671.png: (334800, 54000, 400, 240)
Invalid bounding box for BikesHelmets706.png: (139600, 15000, 400, 300)
Invalid bounding box for BikesHelmets75.png: (172000, 70800, 400, 300)
Invalid bounding box for BikesHelmets75.png: (287600, 72900, 400, 300)
Inva

In [8]:
import xml.etree.ElementTree as ET
import os

def convert_xml_to_yolo(xml_file, output_dir, class_mapping):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    
    output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_file))[0] + '.txt')
    
    with open(output_file, 'w') as f:
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in class_mapping:
                continue
            class_id = class_mapping[class_name]
            
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            x_center = (xmin + xmax) / 2.0 / width
            y_center = (ymin + ymax) / 2.0 / height
            bbox_width = (xmax - xmin) / width
            bbox_height = (ymax - ymin) / height
            
            f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

annotations_dir = r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\annotations"
output_dir = r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLO_annotations"
class_mapping = {"With Helmet": 0, "Without Helmet": 1}

os.makedirs(output_dir, exist_ok=True)

for xml_file in os.listdir(annotations_dir):
    if xml_file.endswith('.xml'):
        convert_xml_to_yolo(os.path.join(annotations_dir, xml_file), output_dir, class_mapping)


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
dataset_dir = 'C:\\Users\\mbsas\\OneDrive\\Dokumenty\\Edge Matrix\\Helmets'
images_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'YOLO_annotations')

output_dir = 'YOLOv5/data'
images_output_dir = os.path.join(output_dir, 'images')
labels_output_dir = os.path.join(output_dir, 'labels')

# Create output directories
os.makedirs(os.path.join(images_output_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(images_output_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(labels_output_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(labels_output_dir, 'val'), exist_ok=True)

# Get list of all images and corresponding labels
images = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
labels = [f.replace('.jpg', '.txt').replace('.png', '.txt') for f in images]

# Split dataset into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Function to copy files to the destination directory
def copy_files(file_list, source_dir, dest_dir):
    for file in file_list:
        src_path = os.path.join(source_dir, file)
        dest_path = os.path.join(dest_dir, file)
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)
        else:
            print(f"File not found: {src_path}, skipping.")

# Copy train images and labels
copy_files(train_images, images_dir, os.path.join(images_output_dir, 'train'))
copy_files(train_labels, labels_dir, os.path.join(labels_output_dir, 'train'))

# Copy validation images and labels
copy_files(val_images, images_dir, os.path.join(images_output_dir, 'val'))
copy_files(val_labels, labels_dir, os.path.join(labels_output_dir, 'val'))

print('Dataset split and copied successfully.')


Dataset split and copied successfully.


In [3]:
import ultralytics

In [2]:
import torch

In [5]:
from ultralytics import YOLO
import torch
import cv2
import numpy as np

In [5]:
model=YOLO('yolov8n.yaml')

In [6]:
results=model.train(data='Yolo project.yaml',epochs=2)

Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=Yolo project.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

train: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\train.cache... 611 images, 3 backgrounds, 13 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     408.72      81.502      71.037      83.004]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     547.68        84.5      132.16         151]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      263.5          43          85          74]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: WARNING  C:\Users\mbsas\OneDrive\


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]


...

        2/2         0G      3.372      4.531      3.775         27        640: 100%|██████████| 38/38 [11:01<00:00, 17.40s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:54<00:00, 10.90s/it]

                   all        151        291   3.05e-05    0.00263    1.6e-05   3.21e-06



2 epochs completed in 0.418 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 6.2MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:47<00:00,  9.50s/it]


                   all        151        291   7.19e-05     0.0105   4.06e-05   4.06e-06
           With Helmet        106        190   0.000144     0.0211   8.12e-05   8.12e-06
        Without Helmet         53        101          0          0          0          0
Speed: 5.3ms preprocess, 283.0ms inference, 0.0ms loss, 9.0ms postprocess per image
Results saved to runs\detect\train5


In [13]:
result=model.train(data='Yolo project.yaml',epochs=10)

Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=Yolo project.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train53, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

train: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\train.cache... 611 images, 3 backgrounds, 13 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     408.72      81.502      71.037      83.004]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     547.68        84.5      132.16         151]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      263.5          43          85          74]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: WARNING  C:\Users\mbsas\OneDrive\


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]
Plotting labels to runs\detect\train53\labels.jpg... 


...

      10/10         0G      1.901      2.241      1.947         11        640: 100%|██████████| 38/38 [09:19<00:00, 14.73s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:52<00:00, 10.41s/it]

                   all        151        291      0.406      0.429      0.398      0.202



10 epochs completed in 1.859 hours.
Optimizer stripped from runs\detect\train53\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train53\weights\best.pt, 6.2MB

Validating runs\detect\train53\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:48<00:00,  9.75s/it]


                   all        151        291      0.405      0.429      0.398      0.202
           With Helmet        106        190      0.552      0.679      0.632      0.337
        Without Helmet         53        101      0.258      0.178      0.163     0.0661
Speed: 5.6ms preprocess, 296.2ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs\detect\train53


In [ ]:
import torch

torch.save(model.state_dict(), 'yolov8n_model_torch.pth')

In [41]:
cap = cv2.VideoCapture(r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\vid1.mp4")

In [7]:
import cv2
import numpy as np
import torch

def preprocess(frame, img_size=640):

    frame_resized = cv2.resize(frame, (img_size, img_size))
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    frame_normalized = frame_rgb / 255.0
    frame_transposed = np.transpose(frame_normalized, (2, 0, 1)).astype(np.float32)
    frame_tensor = torch.from_numpy(frame_transposed)
    
    return frame_tensor


In [43]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_tensor = preprocess(frame).unsqueeze(0)  # Add batch dimension

    # Run the model
    with torch.no_grad():
        detections = model(input_tensor)

    # Post-process detections
    for detection in detections[0]:  # YOLO models usually return a list of detections for each image in the batch
        x, y, w, h = detection[:4].cpu().numpy().astype(int)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 640x640 (no detections), 268.3ms
Speed: 0.0ms preprocess, 268.3ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 292.3ms
Speed: 0.0ms preprocess, 292.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 257.6ms
Speed: 0.0ms preprocess, 257.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 250.1ms
Speed: 0.0ms preprocess, 250.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 253.1ms
Speed: 0.0ms preprocess, 253.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 247.4ms
Speed: 0.0ms preprocess, 247.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 232.1ms
Speed: 0.0ms preprocess, 232.1ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 242.6ms
Speed: 0.0ms pre

AttributeError: 'Results' object has no attribute 'astype'. See valid attributes below.

    A class for storing and manipulating inference results.

    Attributes:
        orig_img (numpy.ndarray): Original image as a numpy array.
        orig_shape (tuple): Original image shape in (height, width) format.
        boxes (Boxes, optional): Object containing detection bounding boxes.
        masks (Masks, optional): Object containing detection masks.
        probs (Probs, optional): Object containing class probabilities for classification tasks.
        keypoints (Keypoints, optional): Object containing detected keypoints for each object.
        speed (dict): Dictionary of preprocess, inference, and postprocess speeds (ms/image).
        names (dict): Dictionary of class names.
        path (str): Path to the image file.

    Methods:
        update(boxes=None, masks=None, probs=None, obb=None): Updates object attributes with new detection results.
        cpu(): Returns a copy of the 

: 

In [10]:
def preprocess_image(image_path, img_size=640):
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (img_size, img_size))
    image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
    image_normalized = image_rgb / 255.0
    image_transposed = np.transpose(image_normalized, (2, 0, 1)).astype(np.float32)
    image_tensor = torch.from_numpy(image_transposed).unsqueeze(0) 
    return image_tensor, image

def visualize_predictions(image, detections):
    for detection in detections:
        x1, y1, x2, y2, conf, cls = map(int, detection[:6].cpu().numpy())
        if conf > 0.25: 
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, f'{cls} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    cv2.imshow('Predictions', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [11]:
image_path = r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\images\BikesHelmets637.png"
input_tensor, original_image = preprocess_image(image_path)


detections = model1(input_tensor)
visualize_predictions(original_image, detections)



0: 640x640 1 With Helmet, 1 Without Helmet, 1110.4ms
Speed: 0.0ms preprocess, 1110.4ms inference, 26.5ms postprocess per image at shape (1, 3, 640, 640)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Results'

In [6]:

model = YOLO('runs/detect/train53/weights/best.pt')
validation_results = model.val(data=r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\Yolo project.yaml")

# The validation_results will contain the metrics
print(validation_results)


Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:37<00:00,  3.79s/it]


                   all        151        291      0.415      0.429      0.396      0.201
           With Helmet        106        190      0.566      0.679      0.629      0.336
        Without Helmet         53        101      0.264      0.178      0.163     0.0655
Speed: 4.2ms preprocess, 227.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\detect\val4
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019E39DF9950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,

In [7]:
# Print the validation results object
print(f"Validation Results: {validation_results}")

Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019E39DF9950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

In [8]:
result=model.train(data='Yolo project.yaml',epochs=5)

Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
engine\trainer: task=detect, mode=train, model=runs/detect/train53/weights/best.pt, data=Yolo project.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

train: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\train.cache... 611 images, 3 backgrounds, 13 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     408.72      81.502      71.037      83.004]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     547.68        84.5      132.16         151]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      263.5          43          85          74]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: WARNING  C:\Users\mbsas\OneDrive\


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]
Plotting labels to runs\detect\train6\labels.jpg... 


...

        5/5         0G      1.929      1.953      1.842         19        640: 100%|██████████| 38/38 [09:51<00:00, 15.56s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:50<00:00, 10.10s/it]

                   all        151        291      0.524      0.536      0.506       0.25



5 epochs completed in 0.880 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 6.2MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:45<00:00,  9.00s/it]


                   all        151        291      0.524      0.536      0.506       0.25
           With Helmet        106        190      0.573      0.784      0.709      0.378
        Without Helmet         53        101      0.475      0.287      0.303      0.122
Speed: 5.3ms preprocess, 274.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train6


In [22]:

model = YOLO('runs/detect/train6/weights/best.pt')
validation_results = model.val(data=r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\Yolo project.yaml")

print(validation_results)


Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:41<00:00,  4.16s/it]


                   all        151        291      0.532      0.541      0.503       0.25
           With Helmet        106        190      0.579      0.784        0.7      0.376
        Without Helmet         53        101      0.484      0.297      0.307      0.124
Speed: 5.2ms preprocess, 251.4ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\val6
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019E511E3ED0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,

In [23]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    original_image = image.copy()
    image = cv2.resize(image, (640, 640))
    image = image / 255.0
    image = torch.from_numpy(image).float().permute(2, 0, 1).unsqueeze(0)
    return image, original_image

def visualize_predictions(image, detections):
    for detection in detections:
        boxes = detection.boxes
        for box in boxes:
            # Extract coordinates, confidence, and class label
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            conf = box.conf[0].cpu().item()
            cls = int(box.cls[0].cpu().item())

            # Draw bounding box if confidence is above a threshold
            if conf > 0.25:
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, f'Class: {cls}, Conf: {conf:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Detections', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [29]:
result=model.train(data='Yolo project.yaml',epochs=10)

Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
engine\trainer: task=detect, mode=train, model=runs/detect/train6/weights/best.pt, data=Yolo project.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

train: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\train.cache... 611 images, 3 backgrounds, 13 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     408.72      81.502      71.037      83.004]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     547.68        84.5      132.16         151]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      263.5          43          85          74]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: WARNING  C:\Users\mbsas\OneDrive\


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]
Plotting labels to runs\detect\train7\labels.jpg... 


...

      10/10         0G        1.6      1.522      1.626         11        640: 100%|██████████| 38/38 [09:41<00:00, 15.29s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:50<00:00, 10.07s/it]

                   all        151        291      0.581      0.588      0.599      0.317



10 epochs completed in 1.771 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 6.2MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:45<00:00,  9.03s/it]


                   all        151        291      0.581      0.587      0.599      0.317
           With Helmet        106        190      0.602      0.789      0.766      0.432
        Without Helmet         53        101       0.56      0.386      0.431      0.201
Speed: 5.2ms preprocess, 275.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train7


In [30]:
result=model.train(data='Yolo project.yaml',epochs=5)

Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
engine\trainer: task=detect, mode=train, model=runs/detect/train6/weights/best.pt, data=Yolo project.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train72, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

train: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\train.cache... 611 images, 3 backgrounds, 13 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     408.72      81.502      71.037      83.004]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     547.68        84.5      132.16         151]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      263.5          43          85          74]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: WARNING  C:\Users\mbsas\OneDrive\


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]
Plotting labels to runs\detect\train72\labels.jpg... 


...

        5/5         0G      1.566       1.41      1.602         11        640: 100%|██████████| 38/38 [09:39<00:00, 15.25s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:50<00:00, 10.08s/it]

                   all        151        291       0.58      0.583      0.607      0.336



5 epochs completed in 0.876 hours.
Optimizer stripped from runs\detect\train72\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train72\weights\best.pt, 6.2MB

Validating runs\detect\train72\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:45<00:00,  9.02s/it]


                   all        151        291       0.58      0.583      0.607      0.336
           With Helmet        106        190      0.545      0.811      0.758      0.445
        Without Helmet         53        101      0.616      0.356      0.456      0.227
Speed: 4.9ms preprocess, 276.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train72


In [12]:
image_path = r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\images\BikesHelmets637.png"
input_tensor, original_image = preprocess_image(image_path)
a=model1.predict(original_image)


0: 448x640 1 With Helmet, 3 Without Helmets, 311.0ms
Speed: 5.5ms preprocess, 311.0ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)


In [13]:

for result in a:
    labels = result.names
    boxes = result.boxes


In [14]:
boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1., 0., 1.])
conf: tensor([0.6328, 0.5194, 0.3252, 0.3044])
data: tensor([[1.8627e+02, 7.6492e+01, 2.0976e+02, 1.0121e+02, 6.3284e-01, 1.0000e+00],
        [4.0231e+01, 8.8721e+01, 6.8735e+01, 1.1996e+02, 5.1938e-01, 1.0000e+00],
        [3.4953e+02, 6.9320e+01, 3.8367e+02, 1.1430e+02, 3.2523e-01, 0.0000e+00],
        [2.4768e+01, 1.0983e+02, 5.2418e+01, 1.4067e+02, 3.0442e-01, 1.0000e+00]])
id: None
is_track: False
orig_shape: (267, 400)
shape: torch.Size([4, 6])
xywh: tensor([[198.0137,  88.8490,  23.4856,  24.7149],
        [ 54.4832, 104.3399,  28.5035,  31.2373],
        [366.5972,  91.8090,  34.1366,  44.9782],
        [ 38.5933, 125.2485,  27.6501,  30.8363]])
xywhn: tensor([[0.4950, 0.3328, 0.0587, 0.0926],
        [0.1362, 0.3908, 0.0713, 0.1170],
        [0.9165, 0.3439, 0.0853, 0.1685],
        [0.0965, 0.4691, 0.0691, 0.1155]])
xyxy: tensor([[186.2709,  76.4915, 209.7565, 101.2065],
        [ 40.2314

In [15]:
for result in a:
    boxes = result.boxes.xyxy  
    scores = result.boxes.conf 
    class_indices = result.boxes.cls  
    labels = result.names  
    image = result.orig_img  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  
    
    for i in range(boxes.shape[0]):
        box = boxes[i].cpu().numpy().astype(int)  
        score = scores[i].item()
        class_index = int(class_indices[i].item()) 
        class_name = labels[class_index]
        pt1 = (box[0], box[1])  
        pt2 = (box[2], box[3])  
        color = (0, 255, 0) 
        thickness = 2  
        image = cv2.rectangle(image, pt1, pt2, color, thickness)
        label = f'{class_name} {score:.2f}'
        image = cv2.putText(image, label, (pt1[0], pt1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

    cv2.imshow('Image with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite('output_image.jpg', image)



In [16]:
image_path=r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\images\BikesHelmets637.png"

image = cv2.imread(image_path)

if image is None:
    print("Error: Image not loaded. Check the file path.")
else:
    print("Image loaded successfully.")
    cv2.imshow('Loaded Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Image loaded successfully.


In [17]:
b=model1.predict(image)



0: 448x640 1 With Helmet, 3 Without Helmets, 351.6ms
Speed: 5.3ms preprocess, 351.6ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)


In [18]:
for result in b:
    boxes = result.boxes.xyxy  
    scores = result.boxes.conf  
    class_indices = result.boxes.cls  
    labels = result.names  
    image = result.orig_img  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    for i in range(boxes.shape[0]):
        box = boxes[i].cpu().numpy().astype(int)  
        score = scores[i].item()
        class_index = int(class_indices[i].item())  
        class_name = labels[class_index]
        pt1 = (box[0], box[1])  
        pt2 = (box[2], box[3])  
        color = (0, 255, 0)  
        thickness = 2  
        image = cv2.rectangle(image, pt1, pt2, color, thickness)
        label = f'{class_name} {score:.2f}'
        image = cv2.putText(image, label, (pt1[0], pt1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)
    cv2.imshow('Image with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite('output_image.jpg', image)



In [19]:
image_path=r"C:\Users\mbsas\Downloads\download.png"

image = cv2.imread(image_path)

if image is None:
    print("Error: Image not loaded. Check the file path.")
else:
    print("Image loaded successfully.")
    cv2.imshow('Loaded Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Image loaded successfully.


In [20]:
c=model1.predict(image)


0: 416x640 1 With Helmet, 1 Without Helmet, 633.0ms
Speed: 8.8ms preprocess, 633.0ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)


In [21]:
colors = {
    0: (0, 255, 0),    
    1: (0, 0, 255),
}


In [22]:
for result in c:
    boxes = result.boxes.xyxy  
    scores = result.boxes.conf
    class_indices = result.boxes.cls  
    labels = result.names  
    image = result.orig_img  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  
    
    for i in range(boxes.shape[0]):
        box = boxes[i].cpu().numpy().astype(int) 
        score = scores[i].item()
        class_index = int(class_indices[i].item()) 
        class_name = labels[class_index]
        pt1 = (box[0], box[1]) 
        pt2 = (box[2], box[3])  
        color = colors.get(class_index, (255, 255, 255))  
        thickness = 2 
        image = cv2.rectangle(image, pt1, pt2, color, thickness)
        label = f'{class_name} {score:.2f}'
        image = cv2.putText(image, label, (pt1[0], pt1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)
    cv2.imshow('Image with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite('output_image.jpg', image)

In [23]:
for result in b:
    boxes = result.boxes.xyxy  
    scores = result.boxes.conf
    class_indices = result.boxes.cls  
    labels = result.names  
    image = result.orig_img  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  
    
    for i in range(boxes.shape[0]):
        box = boxes[i].cpu().numpy().astype(int) 
        score = scores[i].item()
        class_index = int(class_indices[i].item()) 
        class_name = labels[class_index]
        pt1 = (box[0], box[1]) 
        pt2 = (box[2], box[3])  
        color = colors.get(class_index, (255, 255, 255))  
        thickness = 2 
        image = cv2.rectangle(image, pt1, pt2, color, thickness)
        label = f'{class_name} {score:.2f}'
        image = cv2.putText(image, label, (pt1[0], pt1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)
    cv2.imshow('Image with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite('output_image.jpg', image)

In [80]:
result=model.train(data='Yolo project.yaml',epochs=20)

Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
engine\trainer: task=detect, mode=train, model=runs/detect/train6/weights/best.pt, data=Yolo project.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train722, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\train.cache... 611 images, 3 backgrounds, 13 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     408.72      81.502      71.037      83.004]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     547.68        84.5      132.16         151]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      263.5          43          85          74]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: WARNING  C:\Users\mbsas\OneDrive\


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]


...

      20/20         0G      1.381      1.033      1.441         10        640: 100%|██████████| 38/38 [09:45<00:00, 15.41s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:50<00:00, 10.14s/it]

                   all        151        291      0.627      0.684      0.673      0.374



20 epochs completed in 3.637 hours.
Optimizer stripped from runs\detect\train722\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train722\weights\best.pt, 6.2MB

Validating runs\detect\train722\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:46<00:00,  9.22s/it]


                   all        151        291      0.637      0.665      0.677       0.38
           With Helmet        106        190      0.704      0.832      0.804       0.49
        Without Helmet         53        101       0.57      0.498       0.55      0.269
Speed: 5.3ms preprocess, 280.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train722


In [24]:

image_path=r"C:\Users\mbsas\Downloads\HH street.png"

image = cv2.imread(image_path)

if image is None:
    print("Error: Image not loaded. Check the file path.")
else:
    print("Image loaded successfully.")
    cv2.imshow('Loaded Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
d=model1.predict(image)

Image loaded successfully.

0: 448x640 1 With Helmet, 671.3ms
Speed: 28.6ms preprocess, 671.3ms inference, 3.2ms postprocess per image at shape (1, 3, 448, 640)


In [25]:
for result in d:
    boxes = result.boxes.xyxy  
    scores = result.boxes.conf
    class_indices = result.boxes.cls  
    labels = result.names  
    image = result.orig_img  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  
    
    for i in range(boxes.shape[0]):
        box = boxes[i].cpu().numpy().astype(int) 
        score = scores[i].item()
        class_index = int(class_indices[i].item()) 
        class_name = labels[class_index]
        pt1 = (box[0], box[1]) 
        pt2 = (box[2], box[3])  
        color = colors.get(class_index, (255, 255, 255))  
        thickness = 2 
        image = cv2.rectangle(image, pt1, pt2, color, thickness)
        label = f'{class_name} {score:.2f}'
        image = cv2.putText(image, label, (pt1[0], pt1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)
    cv2.imshow('Image with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite('output_image.jpg', image)

In [103]:
result=model.train(data='Yolo project.yaml',epochs=5)


New https://pypi.org/project/ultralytics/8.2.49 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
engine\trainer: task=detect, mode=train, model=runs/detect/train6/weights/best.pt, data=Yolo project.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train7222, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=No

train: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\train.cache... 611 images, 3 backgrounds, 13 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     408.72      81.502      71.037      83.004]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     547.68        84.5      132.16         151]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      263.5          43          85          74]
train: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\train\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: WARNING  C:\Users\mbsas\OneDrive\


val: Scanning C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      379.5        82.5          61          65]
val: WARNING  C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\YOLOv5\data\images\val\BikesHelmets75.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        471       290.5          82         109         761       291.5          84          97       466.5         297         109         118]
Plotting labels to runs\detect\train7222\labels.jpg... 


...

        5/5         0G       1.34     0.9962      1.417         11        640: 100%|██████████| 38/38 [10:47<00:00, 17.03s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:55<00:00, 11.10s/it]

                   all        151        291      0.639      0.684      0.661      0.363



5 epochs completed in 1.329 hours.
Optimizer stripped from runs\detect\train7222\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train7222\weights\best.pt, 6.2MB

Validating runs\detect\train7222\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.11.5 torch-2.3.1+cpu CPU (Intel Core(TM) i5-8350U 1.70GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:48<00:00,  9.77s/it]


                   all        151        291       0.63      0.681      0.652      0.376
           With Helmet        106        190      0.755      0.747      0.786      0.479
        Without Helmet         53        101      0.505      0.614      0.519      0.272
Speed: 5.6ms preprocess, 298.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train7222


In [105]:
video_path =r"C:\Users\mbsas\OneDrive\Dokumenty\Edge Matrix\Helmets\vid1.mp4"
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame) 

    for result in results:
        boxes = result.boxes.xyxy  
        scores = result.boxes.conf 
        class_indices = result.boxes.cls  
        labels = result.names 
        
        for i in range(boxes.shape[0]):
            box = boxes[i].cpu().numpy().astype(int) 
            score = scores[i].item()
            class_index = int(class_indices[i].item())  
            class_name = labels[class_index]
            pt1 = (box[0], box[1])  
            pt2 = (box[2], box[3])  
            color = colors.get(class_index, (255, 255, 255)) 
            thickness = 2  
            frame = cv2.rectangle(frame, pt1, pt2, color, thickness)
            label = f'{class_name} {score:.2f}'
            frame = cv2.putText(frame, label, (pt1[0], pt1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

    out.write(frame)

cap.release()
out.release()



0: 480x640 (no detections), 536.3ms
Speed: 14.5ms preprocess, 536.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 507.0ms
Speed: 15.6ms preprocess, 507.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 596.8ms
Speed: 0.0ms preprocess, 596.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 680.2ms
Speed: 0.0ms preprocess, 680.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 486.2ms
Speed: 9.9ms preprocess, 486.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 439.3ms
Speed: 0.0ms preprocess, 439.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 366.9ms
Speed: 8.1ms preprocess, 366.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 580.4ms
Speed: 7.2ms pre

In [6]:
from ultralytics import YOLO
import torch
import cv2
import numpy as np

In [7]:
model1= YOLO('runs/detect/train7222/weights/best.pt')